### About

This notebook uses the following frameworks
* Langchain

* Langgraph

### 1. Installing dependencies

In [ ]:
!pip install scikit-learn transformers

In [ ]:
!pip install -qU langchain_community langchain-qdrant langchain_huggingface

### 2. Local models

Loading the embedding model "sentence-camembert-large" from hugging face:

In [12]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="dangvantuan/sentence-camembert-large")

No sentence-transformers model found with name dangvantuan/sentence-camembert-large. Creating a new one with mean pooling.


In [13]:
print(f"✓ Model loaded with embedding dimension: {model.get_sentence_embedding_dimension()}")

✓ Model loaded with embedding dimension: 1024


In [61]:
### LLM
from langchain_ollama import ChatOllama

local_llm = "mistral:7b-instruct"
llm = ChatOllama(model=local_llm, temperature=0)
llm_json_mode = ChatOllama(model=local_llm, temperature=0, format="json")

In [62]:
llm.invoke("hi")

AIMessage(content=" Hello! How can I help you today?\n\nIf you have any questions or need assistance with something, feel free to ask. I'm here to help! If you just want to chat, we can talk about anything you like. What would you like to discuss? 😊", additional_kwargs={}, response_metadata={'model': 'mistral:7b-instruct', 'created_at': '2025-05-02T00:26:47.6947382Z', 'done': True, 'done_reason': 'stop', 'total_duration': 15205913000, 'load_duration': 22249500, 'prompt_eval_count': 6, 'prompt_eval_duration': 724269200, 'eval_count': 59, 'eval_duration': 14456600000, 'model_name': 'mistral:7b-instruct'}, id='run-3726b760-26b8-4fdf-b50e-4407659b8881-0', usage_metadata={'input_tokens': 6, 'output_tokens': 59, 'total_tokens': 65})

### 3. Vector store

In [3]:
from qdrant_client import QdrantClient

qdrant = QdrantClient(url="http://localhost:6333")

In [ ]:
import json
import uuid
import hashlib
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, VectorParams, Distance

# Helper function to convert string IDs to valid UUID-compatible IDs
def string_to_uuid(string_id):
    """Convert a string to a deterministic UUID by hashing it"""
    # Create MD5 hash of the string
    hash_object = hashlib.md5(string_id.encode())
    # Convert to hex
    hex_dig = hash_object.hexdigest()
    # Create a UUID from the hex string
    return uuid.UUID(hex_dig)

In [ ]:
# 4. Load your JSON file
try:
    with open('output_chunks.json', 'r', encoding='utf-8') as f:
        documents = json.load(f)
    print(f"✓ Loaded {len(documents)} documents from output_chunks.json")
except FileNotFoundError:
    print("Error: output_chunks.json file not found!")
    sys.exit(1)
except json.JSONDecodeError:
    print("Error: Invalid JSON format in output_chunks.json!")
    sys.exit(1)

# 5. Create a collection
collection_name = "Auditron_legal_chunks"
print(f"Creating collection '{collection_name}'...")
qdrant.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=model.get_sentence_embedding_dimension(), distance=Distance.COSINE),
)

# 6. Process the documents
print("Processing documents...")
total_chunks = sum(len(doc.get("chunks", [])) for doc in documents)
batch_size = 50
processed_count = 0
error_count = 0
batch_points = []
id_mapping = {}  # To store mapping between original IDs and UUID IDs

for doc_idx, doc in enumerate(documents):
    print(f"Processing document {doc_idx+1}/{len(documents)}")
    chunks = doc.get("chunks", [])
    
    for chunk_idx, chunk in enumerate(chunks):
        try:
            # Extract text and ID
            text = chunk.get("text", "")
            original_id = chunk.get("chunk_id", f"unknown_{doc_idx}_{chunk_idx}")
            
            if not text.strip():  # Skip empty chunks
                print(f"Warning: Empty text in chunk {original_id}")
                continue
            
            # Clean and truncate text if necessary to avoid index errors
            # Some models have maximum input length limitations
            max_text_length = 512  # Adjust based on your model's limitations
            text = text.strip()[:max_text_length]
            
            # Convert string ID to UUID
            point_id = string_to_uuid(original_id)
            
            # Save mapping
            id_mapping[str(point_id)] = original_id
            
            # Generate embedding with error handling
            try:
                embedding = model.encode(text, show_progress_bar=False)
            except Exception as embed_error:
                print(f"Embedding error for chunk {original_id}: {str(embed_error)}")
                # Try with a shorter text if it might be a length issue
                if len(text) > 200:
                    try:
                        shorter_text = text[:200]
                        print(f"Retrying with shorter text for {original_id}")
                        embedding = model.encode(shorter_text, show_progress_bar=False)
                    except Exception as retry_error:
                        print(f"Still failed with shorter text: {str(retry_error)}")
                        error_count += 1
                        continue
                else:
                    error_count += 1
                    continue
            
            # Create point with UUID
            point = PointStruct(
                id=str(point_id),
                vector=embedding.tolist(),
                payload={
                    "text": text,
                    "original_id": original_id,  # Keep original ID in payload
                    "structures": chunk.get("structures", []),
                    "document_path": chunk.get("document_path", []),
                    "metadata": chunk.get("metadata", {})
                }
            )
            
            # Add to batch
            batch_points.append(point)
            processed_count += 1
            
            # If batch is full, upload to Qdrant
            if len(batch_points) >= batch_size:
                qdrant.upsert(
                    collection_name=collection_name,
                    points=batch_points,
                )
                print(f"Uploaded batch: {processed_count}/{total_chunks} chunks ({error_count} errors so far)")
                batch_points = []
                
        except Exception as e:
            print(f"Error processing chunk {original_id}: {str(e)}")
            error_count += 1

# Upload any remaining points
if batch_points:
    qdrant.upsert(
        collection_name=collection_name,
        points=batch_points,
    )
    print(f"Uploaded final batch: {processed_count}/{total_chunks} chunks")

# Save ID mapping for reference (optional)
try:
    with open('id_mapping.json', 'w', encoding='utf-8') as f:
        json.dump(id_mapping, f, indent=2)
    print("✓ Saved ID mapping to id_mapping.json")
except Exception as e:
    print(f"Warning: Could not save ID mapping: {str(e)}")

print(f"✅ Successfully processed {processed_count}/{total_chunks} chunks into Qdrant collection '{collection_name}'!")
print(f"Total errors encountered: {error_count}")

### 4. Retrieving

In [37]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import SearchParams
from langchain_core.embeddings import Embeddings

# 3. Create the vector store with LangChain
vector_store = QdrantVectorStore(
    client=qdrant,
    collection_name="Auditron_legal_chunks",  # Your collection name
    content_payload_key="text", 
    embedding=embeddings,
)
#    search_params=SearchParams(hnsw_ef=128)  # Your search params
# 4. Create the retriever from the vector store
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

# 5. Use the retriever
query = "Quel est le nouveau taux unifié de retenue à la source applicable aux loyers, rémunérations non commerciales, honoraires et commissions en Tunisie depuis l'adoption de la loi N° 2020-46 du 23 décembre 2020?"
documents = retriever.invoke(query)

# The retrieved documents will include content and metadata
for doc in documents:
    print(doc.page_content)  # Access the content
    print(doc.metadata)      # Access the metadata

Pour la retenue à la source au titre des revenus de capitaux mobiliers L’article 17 de la loi n°2020-46 du 23 décembre 2020 portant loi de finances pour l'année 2021 a révisé le régime fiscal des revenus de capitaux mobiliers, et ce, par la généralisation de l'application de la retenue à la source libératoire au taux de 20% à tous les revenus de capitaux mobiliers. Ladite retenue à la source est définitive et non susceptible de déduction ou de restitution, et ce, nonobstant le régime fiscal et le résultat q
{'doc_type': 'notes_communes', 'doc_year': 2022, 'doc_title': 'note_commune_01_2022', 'doc_language': 'fr', 'token_count': 247, 'source_url': 'https://jibaya.tn/wp-content/uploads/2024/02/Note-Commune-N%C2%B001-1.pdf', '_id': '2f29e3bf-69cb-9211-4d32-ca2a1d5e5554', '_collection_name': 'Auditron_legal_chunks'}
Pour la retenue à la source Suite à l'augmentation du taux de l'impôt sur les sociétés de 15% à 20%, les taux de retenue à la source dus au titre des opérations de cession par 

### Document re-ranking

In [55]:
import json
from langchain_core.messages import HumanMessage, SystemMessage

### Retrieval Grader

# Doc grader instructions
doc_grader_instructions = """Vous êtes un modèle chargé d’évaluer la pertinence d’un document récupéré par rapport à une question utilisateur.

Si le document contient des mots-clés ou un sens sémantique lié à la question, considérez-le comme pertinent."""

# Grader prompt
doc_grader_prompt = """Voici le document récupéré : \n\n {document} \n\n Voici la question de l'utilisateur : \n\n {question}.

Évaluez soigneusement et objectivement si le document contient au moins une information pertinente en lien avec la question.

Retournez un JSON avec une clé unique, binary_score, qui sera 'oui' ou 'non' pour indiquer si le document contient au moins une information pertinente pour la question."""
# Test
question = "Quel est le nouveau taux unifié de retenue à la source applicable aux loyers, rémunérations non commerciales, honoraires et commissions en Tunisie depuis l'adoption de la loi N° 2020-46 du 23 décembre 2020?"

docs = retriever.invoke(question)
doc_txt = docs[0].page_content

doc_grader_prompt_formatted = doc_grader_prompt.format(
    document=doc_txt, question=question
)
result = llm_json_mode.invoke(
    [SystemMessage(content=doc_grader_instructions)]
    + [HumanMessage(content=doc_grader_prompt_formatted)]
)
json.loads(result.content)

{'binary_score': 'non'}

In [56]:
doc_txt

"Pour la retenue à la source au titre des revenus de capitaux mobiliers L’article 17 de la loi n°2020-46 du 23 décembre 2020 portant loi de finances pour l'année 2021 a révisé le régime fiscal des revenus de capitaux mobiliers, et ce, par la généralisation de l'application de la retenue à la source libératoire au taux de 20% à tous les revenus de capitaux mobiliers. Ladite retenue à la source est définitive et non susceptible de déduction ou de restitution, et ce, nonobstant le régime fiscal et le résultat q"

In [49]:
docs = retriever.invoke(question)
doc_txt = docs[2].page_content

doc_grader_prompt_formatted = doc_grader_prompt.format(
    document=doc_txt, question=question
)
result = llm_json_mode.invoke(
    [SystemMessage(content=doc_grader_instructions)]
    + [HumanMessage(content=doc_grader_prompt_formatted)]
)
json.loads(result.content)

{'binary_score': 'oui'}

In [50]:
doc_txt

'10%(1) au titre des honoraires, commissions, courtages, loyers et rémunérations des activités non commerciales qu’elle qu’en soit l’appellation payés par l’Etat, les collectivités locales, les personnes morales ainsi que les personnes physiques soumises à l’impôt sur le revenu selon le régime réel et les personnes visées au paragraphe II de l’article 22 du présent code. (Modifié Art 69-1 LF 2004-90 du 31/12/2004, Art.45-1 LF 2012-27 du 29/12/2012 et Art 14-5 LF 2020-46 du 23/12/2020). (1) Ce taux s’applique'

### 5. Generation

In [58]:
### Generate

# Prompt
rag_prompt = """Vous êtes un assistant pour des tâches de question-réponse.

Voici le contexte à utiliser pour répondre à la question :

{context}

Réfléchissez soigneusement au contexte ci-dessus.

Maintenant, examinez la question de l'utilisateur :

{question}

Fournissez une réponse à cette question en utilisant uniquement le contexte ci-dessus.

Utilisez un maximum de trois phrases et gardez la réponse concise.

Réponse :"""

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# Test
docs = retriever.invoke(question)
docs_txt = format_docs(docs)
rag_prompt_formatted = rag_prompt.format(context=docs_txt, question=question)
generation = llm.invoke([HumanMessage(content=rag_prompt_formatted)])
print(generation.content)


 Le nouveau taux unifié de retenue à la source applicable aux loyers, rémunérations non commerciales, honoraires et commissions en Tunisie est de 20%.


Hallucination grader:

In [57]:
### Hallucination Grader

# Hallucination grader instructions
hallucination_grader_instructions = """

Vous êtes un enseignant en train de corriger un quiz.

Vous recevrez des FAITS et une RÉPONSE D'ÉLÈVE.

Voici les critères de notation à suivre :

(1) Assurez-vous que la RÉPONSE DE L'ÉLÈVE est bien fondée sur les FAITS.

(2) Assurez-vous que la RÉPONSE DE L'ÉLÈVE ne contient pas d'informations « hallucinées » qui sortent du cadre des FAITS.

Note :

Une note de oui signifie que la réponse de l'élève respecte tous les critères. C’est la note la plus élevée (meilleure).

Une note de non signifie que la réponse de l'élève ne respecte pas tous les critères. C’est la note la plus basse que vous pouvez attribuer.

Expliquez votre raisonnement étape par étape afin de garantir la justesse de votre raisonnement et de votre conclusion.

Évitez d’énoncer directement la bonne réponse dès le départ.
"""


# Grader prompt
hallucination_grader_prompt = """FAITS : \n\n {documents} \n\n RÉPONSE DE L'ÉLÈVE : {generation}.

Retournez un JSON avec deux clés :  
- binary_score : une valeur 'oui' ou 'non' indiquant si la RÉPONSE DE L'ÉLÈVE est bien fondée sur les FAITS.  
- explanation : une explication justifiant la note attribuée.
"""

# Test using documents and generation from above
hallucination_grader_prompt_formatted = hallucination_grader_prompt.format(
    documents=docs_txt, generation=generation.content
)
result = llm_json_mode.invoke(
    [SystemMessage(content=hallucination_grader_instructions)]
    + [HumanMessage(content=hallucination_grader_prompt_formatted)]
)
json.loads(result.content)

{'binary_score': 'oui',
 'explanation': "La réponse de l'élève est correcte car elle indique que le nouveau taux unifié de retenue à la source applicable aux loyers, rémunérations non commerciales, honoraires et commissions en Tunisie est de 10%. Cette information est bien fondée sur les faits fournis qui mentionnent que le taux de retenue à la source a été généralisé à 10% pour ces catégories de revenus par l'article 14-5 LF 2020-46 du 23 décembre 2020."}

Answer grader

In [60]:
### Answer Grader

# Answer grader instructions
answer_grader_instructions = """Vous êtes un enseignant en train de corriger un quiz.

Vous recevrez une QUESTION et une RÉPONSE D'ÉLÈVE.

Voici les critères de notation à suivre :

(1) La RÉPONSE DE L'ÉLÈVE aide à répondre à la QUESTION.

Note :

Une note de oui signifie que la réponse de l'élève respecte tous les critères. C’est la note la plus élevée (meilleure).

L’élève peut recevoir une note de oui même si la réponse contient des informations supplémentaires qui ne sont pas explicitement demandées dans la question.

Une note de non signifie que la réponse de l'élève ne respecte pas tous les critères. C’est la note la plus basse que vous pouvez attribuer.

Expliquez votre raisonnement étape par étape afin de garantir la justesse de votre raisonnement et de votre conclusion.

Évitez d’énoncer directement la bonne réponse dès le départ.
"""

# Grader prompt
answer_grader_prompt = """QUESTION : \n\n {question} \n\n RÉPONSE DE L'ÉLÈVE : {generation}.

Retournez un JSON avec deux clés :  
- binary_score : une valeur 'oui' ou 'non' indiquant si la RÉPONSE DE L'ÉLÈVE respecte les critères.  
- explanation : une explication justifiant la note attribuée.
"""


# Test
answer = generation.content

# Test using question and generation from above
answer_grader_prompt_formatted = answer_grader_prompt.format(
    question=question, generation=answer
)
result = llm_json_mode.invoke(
    [SystemMessage(content=answer_grader_instructions)]
    + [HumanMessage(content=answer_grader_prompt_formatted)]
)
json.loads(result.content)

{'binary_score': 'oui',
 'explanation': "La réponse de l'élève est correcte car elle indique le nouveau taux unifié de retenue à la source applicable aux loyers, rémunérations non commerciales, honoraires et commissions en Tunisie depuis l'adoption de la loi N° 2020-46 du 23 décembre 2020. Elle respecte tous les critères demandés."}

### 6. ChatAgent with LangGraph